In [ ]:
%%html
<style>
.dataframe th {
    font-size: 18px;
}
.dataframe td {
    font-size: 20px;
}
</style>

In [ ]:
import sys 
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.stats import sigma_clip
from glob import glob
from os.path import basename
import matplotlib.colors
sys.path.append('/data/des81.b/data/emarrufo/AstroSkipper_Analysis/astroskip/astroskip/image')
sys.path.append('/data/des81.b/data/emarrufo/AstroSkipper_Analysis/full_CCD_characterization/')
sys.path.append('/data/des81.b/data/emarrufo/AstroSkipper_Analysis/full_CCD_characterization/ccd-charge-diffusion-master')
from skipper_image import *
import peak_finding_algorithm 
import fit_peaks
import make_abs_calibration
import combined
import dratio
import warnings
warnings.filterwarnings("ignore")
from natsort import natsorted 
import pandas as pd
from IPython.display import display, Markdown
from matplotlib.font_manager import FontProperties
import matplotlib.font_manager as font_manager
matplotlib.font_manager._rebuild()
import matplotlib.colors
import scipy
import cp_pipe_functions
from matplotlib.font_manager import FontProperties
import matplotlib.font_manager as font_manager
from scipy.optimize import curve_fit

# Data 
Here we load the data products for charaterizing the AstroSkipper detectors. Data taking procedure and products definition can be located here https://docs.google.com/document/d/1JaMrdpenERU0nBifb3hPdRZIRxmdC5SqRKWDj2JUhgk/edit

In [ ]:
# Path to data files

# Gain Calculation

In [ ]:
def get_gain(image,plot=True,disp=True):
    hdus=4
    img = SkipperImage(image)
    img_os = img.getImages()
    gain=dict()
    gain_arr=list()
    for i in range(hdus):
        clipped_img_os = sigma_clip(img_os[i].flatten(),sigma=3)
        _, _, k =peak_finding_algorithm.get_peaks(clipped_img_os, plot=plot)
        
        gain_arr.append(k)
    
    gain_arr=np.array(gain_arr)
    gain_arr=np.nan_to_num(gain_arr)
    m = np.median(gain_arr[gain_arr > 0])
    gain_arr[gain_arr == 0] = m
    
    for i in range(hdus):
        gain.update({i:[gain_arr[i]]})

    if disp:
        df = pd.DataFrame(data=gain)
        df.style.set_table_attributes('style="font-size: 17px"')
        
        title="Gain Measurments (ADUs/e-)"
        display(Markdown('<strong>{}'.format(title)))
        display(df)
    
    return np.array(gain_arr)


# VSUB (Skipping)

In [ ]:
def test_vsub_skipping(files,offset=0.4):
    hdus=4
    gain = get_gain(bckg[-1],plot=False,disp=False)
    for f in files:
        img = SkipperImage(f)
        img_os = np.array(img.getImages())
        for hdu in range(hdus):
            hdr=fits.open(f)[hdu+1].header
            VSUB=hdr['VSUB']
            msg = "VSUB={}".format(VSUB)
            msg_hdu="HDU={}".format(hdu)
            display(Markdown('<strong>{}'.format(msg_hdu)))
            display(Markdown('<strong>{}'.format(msg)))
            img_os_clipped =  sigma_clip(img_os[hdu].flatten(),sigma=3.5)/gain[hdu] + offset 
            fit_peaks.fit_multi_gaussian(img_os_clipped,-2,7,plot=True)
    

            
#test_vsub_skipping(nsamp_sample_darks_vsub)

In [ ]:
def test_vr_skipping(files,offset=0.8):
    hdus=4
    gain = get_gain(bckg[-1],plot=False,disp=False)
    for f in files:
        img = SkipperImage(f)
        img_os = np.array(img.getImages())
        for hdu in range(hdus):
            hdr=fits.open(f)[hdu+1].header
            VR=hdr['VR']
            msg = "VR={}".format(VR)
            msg_hdu="HDU={}".format(hdu)
            display(Markdown('<strong>{}'.format(msg_hdu)))
            display(Markdown('<strong>{}'.format(msg)))
            img_os_clipped =  sigma_clip(img_os[hdu].flatten(),sigma=3.5)/gain[hdu] + offset 
            fit_peaks.fit_multi_gaussian(img_os_clipped,-2,7,plot=True)

In [ ]:
def single_sample_bias(files, bckg_img,plot=True,disp=True):
    hdus=4
    median=list()
    noise=list()
    OSS=3080
    gain = get_gain(bckg[-1],plot=False,disp=False)
    
    for hdu in range(hdus):
        for file in files:
            img=fits.open(file)
            full_img = sigma_clip(img[hdu].data[:,:].flatten(),maxiters=None) / gain[hdu]
            OS=sigma_clip(img[hdu].data[:,OSS:].flatten(),maxiters=None) / gain[hdu]
            median.append(np.median(full_img))
            noise.append(np.std(OS))   
    if plot:
        font = FontProperties()
        #font.set_name('Cambria Math')
        #plt.rcParams["font.family"] = "Cambria Math"
        median=np.array(median).reshape(hdus,len(files))
        noise=np.array(noise).reshape(hdus,len(files))
        fig1, ax1 = plt.subplots(figsize=(8, 5))
        for i in range(hdus):
            plt.scatter(np.arange(0,len(files),1),median[i],s=50,label="HDU={}".format(i))
        
        plt.grid()
        ax1.set_ylabel('Signal Median, $\mu_{S}$ (e-/pix)', fontsize=30)
        ax1.set_xlabel('Image Number',fontsize=30)
        ax1.tick_params(axis='both', which='major', labelsize=25)
        plt.legend(loc='best',fontsize=15)
        plt.show()
        
        fig2, ax2 = plt.subplots(figsize=(8, 5))
        for i in range(hdus):
            plt.scatter(np.arange(0,len(files),1),noise[i],s=50, marker="x",label="HDU={}".format(i))
        
        plt.grid()
        ax2.set_ylabel('Signal Noise, $\sigma_{S}$ (e- rms/pix)', fontsize=30)
        ax2.set_xlabel('Image Number',fontsize=30)
        ax2.tick_params(axis='both', which='major', labelsize=25)
        plt.legend(loc='best',fontsize=15)
        plt.show()
    
    if disp:
        noise=np.array(noise).reshape(hdus,len(files))
        avg_noise=np.mean(noise,axis=1)
        noise_dict=dict()
        
        for i in range(hdus):
            noise_dict.update({i:[avg_noise[i]]})
        
        df = pd.DataFrame(data=noise_dict)
        df.style.set_table_attributes('style="font-size: 17px"')
        title = "Average Noise/HDU (e- rms/pix)"
        display(Markdown('<strong>{}'.format(title)))
        display(df)

In [ ]:
def background(file,bckg_img,offset=0,plot=False):
    hdus=4
    img = SkipperImage(file)
    img_os = img.getOverscans()
    img_active = img.getImages()
    gain = get_gain(bckg_img,plot=False,disp=False)
    median=list()
    for i in range(hdus):
        clipped_img_active = sigma_clip(img_active[i].flatten(),sigma=3.5)/gain[i] + offset
        median.append(np.median(clipped_img_active))
        if plot:
            fit_peaks.fit_multi_gaussian(clipped_img,-1,5,plot=False)
    
    return np.array(median)

def sp_charge(file,bckg_img,offset=0.8,plot=False):
    hdus=4
    img = SkipperImage(file)
    img_os = img.getOverscans()
    img_active = img.getImages()
    gain = get_gain(bckg_img,plot=False,disp=False)
    spurios=dict()
    b=background(bckg_img,bckg_img,offset=0,plot=False)
    b=b[b >= 0]
    b_average=np.mean(b)
    for i in range(hdus):
        clipped_img_active = sigma_clip(img_active[i].flatten(),sigma=3.5)/gain[i] + offset
        spurios.update({i:[np.median(clipped_img_active) - b_average]})

        
        if plot:
            fit_peaks.fit_multi_gaussian(clipped_img_active,-1,8,plot=plot)
    
    df = pd.DataFrame(data=spurios)
    df.style.set_table_attributes('style="font-size: 17px"')
    title = "Spurious  Charge/HDU (e- rms/pix)"
    display(Markdown('<strong>{}'.format(title)))
    display(df)
    
#sp_charge(nsamp_biases_sp [-2],bckg[-1])

In [ ]:
def dark_current(files,bckg_img):
    hdus=4
    exp_time=400
    out_combined_dark="combined_dark.fits"
    combined.make_combined_img(files,out_combined_dark)
    combined_dark = fits.open(out_combined_dark)
    median=list()
    gain= get_gain(bckg_img,plot=False,disp=False)
    dc=dict()

    for hdu in range(hdus):
        data_dark = sigma_clip(combined_dark[hdu].data[:,:].flatten(),maxiters=None,sigma=3)
        dc.update({hdu:[np.median(data_dark)/gain[hdu]/exp_time]})
    
    df = pd.DataFrame(data=dc)
    df.style.set_table_attributes('style="font-size: 17px"')
    title = "Dark Current (e-/pix/sec)"
    display(Markdown('<strong>{}'.format(title)))
    display(df)

In [ ]:
def bad_pixels_dark(files,plot=False,plot_mask=True):
    out_combined_dark="combined_dark.fits"
    combined.make_combined_img(files,out_combined_dark)
    amps=4
    image = fits.open(out_combined_dark)
    #image = fits.open(files[0])
    img_shape=image[1].shape
    bad_pix_percent=dict()
    
    
    msg="Darks(bright pixels)"
    display(Markdown('<strong>{}'.format(msg)))
    
    if plot:
        for i in range(amps):
            img=(image[i].data.flatten())
            plt.figure(figsize=(5,5))
            plt.hist(img,bins=np.linspace(np.min(img),np.max(img),150),
                     histtype='step',density=False,color='orange')
            plt.yscale('log')
            plt.grid()
            plt.show()
    
    for i in range(amps):
        masked_img = sigma_clip(image[i].data.flatten(),masked=True,sigma=4)
        mask=masked_img.mask.reshape(img_shape)
        total_pixels = len(masked_img)
        num_bad_pixels =len(np.where(masked_img.mask==True)[0])
        percent_bad =  num_bad_pixels/total_pixels
        bad_pix_percent.update({i:[percent_bad]})

        
        if plot_mask:
            cmap = matplotlib.colors.ListedColormap(['black', 'white'])
            plt.figure(figsize=(15,15))
            plt.imshow(mask, cmap=cmap)
            #plt.colorbar()
            plt.show()
        
    df = pd.DataFrame(data=bad_pix_percent)
    df.style.set_table_attributes('style="font-size: 17px"')
    title = "Percentage of bad pixels/HDU"
    display(Markdown('<strong>{}'.format(title)))
    display(df)
        
        
        
    return mask


# Light Tests

In [ ]:
def bad_pixels_flat(files,plot_mask=True):
    out_combined_flat="combined_flat.fits"
    combined.make_combined_img(files,out_combined_flat)
    amps=4
    OSS=3079
    image = fits.open(out_combined_flat)
    img_shape=image[1].shape
    bad_pix_percent=dict()
    
    msg="Flats(dead pixels)"
    display(Markdown('<strong>{}'.format(msg)))
    
    for i in range(amps):
        
        masked_img = sigma_clip(image[i].data.flatten(),masked=True,sigma=3.5)
        mask=masked_img.mask.reshape(img_shape)
        total_pixels =len(masked_img)
        num_bad_pixels =len(np.where(masked_img.mask==True)[0])
        percent_bad = num_bad_pixels/total_pixels
        bad_pix_percent.update({i:[percent_bad]})

        
        if plot_mask:
            cmap = matplotlib.colors.ListedColormap(['black', 'white'])
            plt.figure(figsize=(15,15))
            plt.imshow(mask, cmap=cmap)
            #plt.colorbar(orientation="horizontal", pad=0.05)
            plt.show()      
            
    df = pd.DataFrame(data=bad_pix_percent)
    title= "Percentage of bad pixels/HDU"
    display(Markdown('<strong>{}'.format(title)))
    display(df)
    
    return mask

In [ ]:
# Linearity 
def linearity(files,time_idx,plot=True):
    f_line = lambda x, *p: p[0] * x + p[1]
    def nonlinear(params,signal,time):
        model = f_line(np.array(time),*params)
        deviation = signal - model 
        non_linearity = (np.max(deviation) + np.min(deviation)) / np.max(signal)
        return  np.abs(non_linearity)
    
    hdus=4
    OSS=3079
    signal_mean=list()
    time=list()
    
    for f in files:
        time.append(float(basename(f).split('_')[time_idx]))
        
    for hdu in range(hdus):
        for f in files:
            img=fits.open(f)
            full_img = sigma_clip(img[hdu].data[:,:OSS].flatten(),maxiters=None)
            signal_mean.append(np.median(full_img))
    
    signal_mean = np.array(signal_mean).reshape(hdus,len(files))
    signal_mean = np.nan_to_num(signal_mean)
    
    if plot:
        font = FontProperties()
        font.set_name('Cambria Math')
        for i in range(hdus):
            params,cov = scipy.optimize.curve_fit(f_line, xdata=np.array(time), ydata=signal_mean[i], p0=(20000,40))
            y_fit = f_line(np.array(time),*params)
            nl=nonlinear(params,signal_mean[i],time)
            fig1, ax1 = plt.subplots(figsize=(8, 8))
            plt.scatter(time,signal_mean[i],s=100,marker = "x",label="HDU={}".format(i))
            plt.plot(time, y_fit, '--', color='red',linewidth=3)
            plt.grid()
            plt.title("Non-linearity = {:.3}".format(nl),fontsize=30)
            ax1.set_ylabel('Signal Mean, $\mu_{S}$ (ADU)', fontsize=35)
            ax1.set_xlabel('Exposure Time(s)',fontsize=35)
            ax1.tick_params(axis='both', which='major', labelsize=30)
            plt.legend(loc='best',fontsize=20)
            plt.show()


# PTC

In [ ]:
# Creates a dictionary with path to each image where trial 
def FileDict(files):
    fileArray=dict()
    for f in files:
        time=float(basename(f).split('_')[10])
        trial=int(basename(f).split('_')[13])
        if(not (time in fileArray.keys())):
            fileArray[time]=dict()
        fileArray[time][trial]=f
    return fileArray

# gets the exposure times 
def getTimes(f):
    timesAsc=np.sort(list(FileDict(f).keys()))
    return timesAsc

In [ ]:
def ptc(f,sigma=3.5):
    hdus=4
    OSS=3100
    skipRow=5
    skipCol=2000
    noiseStds=np.zeros(shape=(hdus,len(getTimes(f))))
    mu=np.zeros(shape=(hdus,len(getTimes(f))))
    signalDiffVar=np.zeros(shape=(hdus,len(getTimes(f))))
    gain=get_gain(bckg[-1],plot=False,disp=False)

    for hdu in range(hdus):
        for i in range(0,len(getTimes(f))):
            timeFiles=FileDict(f)[getTimes(f)[i]]
            trial1=timeFiles[1]
            trial2=timeFiles[2]
            full_image=fits.open(trial2)
            OS=sigma_clip(full_image[hdu].data[skipRow:,OSS:].flatten(),maxiters=None,sigma=sigma)
            
            img1=fits.open(trial1)
            img1=sigma_clip(img1[hdu].data[skipRow:,skipCol:OSS].flatten(),maxiters=None,sigma=sigma)
            
            img2=fits.open(trial2)
            img2=sigma_clip(img2[hdu].data[skipRow:,skipCol:OSS].flatten(),maxiters=None,sigma=sigma)
            
            mu_1 = np.ma.mean(img1)
            mu_2 =np.ma.mean(img2)
            mu[hdu][i]=(mu_1 + mu_2)/2.0
            img1=img1-np.ma.median(img1)
            img2=img2-np.ma.median(img2)
            
            noiseStds[hdu][i]=np.ma.std(sigma_clip(OS.flatten(),maxiters=None,sigma=sigma))
            
            imgDiff = (mu_2*img1 - mu_1*img2)/mu[hdu][i]
            varDiff = np.ma.var(imgDiff)/2
            varFactor = cp_pipe_functions.sigmaClipCorrection(sigma)**2
            varDiff*= varFactor
            signalDiffVar[hdu][i] = varDiff
    avg_noise = np.median(noiseStds,axis=1)
    for hdu in range(hdus):
       
        initialParams = [-1e-6,1./gain[hdu], avg_noise[hdu]**2] 
        polyFit, polyFitErr, chiSq, weightsY = cp_pipe_functions.irlsFit(initialParams, mu[hdu][7:-23],
                                                                         signalDiffVar[hdu][7:-23],
                                                                         cp_pipe_functions.funcAstier)
        x_fit = np.arange(np.min(mu[hdu]),np.max(mu[hdu]),100)
        y_fit =  cp_pipe_functions.funcAstier(polyFit, x_fit)
        gain_hdu = 1/polyFit[1]
        
        fig, ax = plt.subplots(figsize=(10, 10))
        max_mean=np.max(mu[hdu])
        
        textstr = '\n'.join((
            r'$FW=%.0f$ e-' % (max_mean/gain_hdu, ),
            r'$a_{00}=%.2e$' % (polyFit[0], ),
            r'$gain=%.2f$ ADU/e-' % (gain_hdu, ),
            r'$\sigma=%.2f$ e- rms/pix' % (avg_noise[hdu]/gain_hdu , )))
        
        props = dict(boxstyle='square', facecolor='white', alpha=0.2)
        ax.text(0.05, 0.75, textstr, transform=ax.transAxes, fontsize=20,
                verticalalignment='top', bbox=props)

        plt.scatter(mu[hdu],signalDiffVar[hdu], s=100, marker="X", label='Signal')
        plt.plot(x_fit,y_fit,'--r',label="Model Fit")
        plt.scatter(max_mean,cp_pipe_functions.funcAstier(polyFit,max_mean), marker="*", label = "Full Well",s=700)
        ax.set_xlabel('Signal Mean, $\mu_{S}$ (ADU)', fontsize=35)
        ax.set_ylabel('Signal Variance, $\sigma^{2}_{S}$ (ADU)',fontsize=35)
        ax.tick_params(axis='both', which='major', labelsize=30)
        plt.legend(loc='best',fontsize=20)
        plt.grid()
        plt.yscale('log')
        plt.xscale('log')
        plt.show()
    
#ptc(ptc_files,sigma=3.5)

# CTI 


In [ ]:
def cti(files):
    hdus=4
    nTransfers=3200
    row_min=10
    row_max=50
    first_OS=3080
    OSS=3079
    CTI=np.zeros(shape=(hdus,len(getTimes(files))))
    mu=np.zeros(shape=(hdus,len(getTimes(files))))
    CTI_dict=dict()
    gain=get_gain(bckg[-1],plot=False,disp=False)
    skipRow=5
    skipCol=2000
    marker =["p","v","*",'X']
    
    for hdu in range(hdus):
        for i in range(0,len(getTimes(files))):
            timeFiles=FileDict(files)[getTimes(files)[i]]
            trial1=timeFiles[1]
            trial2=timeFiles[2]
            
            img1=fits.open(trial1)
            img2=fits.open(trial2)            
            
            dc_1 = np.mean((img1[hdu].data[row_min:row_max,first_OS]))/gain[hdu]
            dc_2 = np.mean((img2[hdu].data[row_min:row_max,first_OS]))/gain[hdu]
            
            
            img1_data=sigma_clip(img1[hdu].data[skipRow:,skipCol:OSS].flatten(),maxiters=None)
            img2_data=sigma_clip(img2[hdu].data[skipRow:,skipCol:OSS].flatten(),maxiters=None)
            
            mu_1 = np.ma.mean(img1_data)
            mu_2 =np.ma.mean(img2_data)
            mu[hdu][i]=(mu_1 + mu_2)/2.0
            
            lc_1= np.mean((img1[hdu].data[row_min:row_max,OSS]))/gain[hdu]
            lc_2= np.mean((img2[hdu].data[row_min:row_max,OSS]))/gain[hdu]

        
            CTI_1 = np.abs(dc_1/(lc_1*nTransfers))
            CTI_2 = np.abs(dc_2/(lc_2*nTransfers))
            CTI[hdu][i] = (CTI_1+CTI_2)/2
                        
    CTI_median = np.median(CTI,axis=1)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    for hdu in range(hdus):
        CTI_dict.update({hdu:[CTI_median[hdu]]})
        plt.scatter(mu[hdu]/gain[hdu],CTI[hdu],s=150,alpha=1 ,label = "HDU={}".format(hdu), marker=marker[hdu])

    ax.tick_params(axis='both', which='major', labelsize=15)
    plt.xlabel(r"$\rm Signal (e^{-}/pix)$",fontsize=40)
    plt.ylabel("CTI",fontsize=40)
    plt.legend(loc='best',fontsize=20)
    plt.ylim(-1e-9,1e-7)
    plt.grid()
    plt.show()
        
    
    df = pd.DataFrame(data=CTI_dict)
    title="Median CTI/HDU"
    display(Markdown('<strong>{}'.format(title)))
    display(df)

# Absolute QE

In [ ]:
def calculate_abs_qe(proc_qe_files,raw_qe_files,exp_time=4.2):
    hdus=4
    gain=get_gain(bckg[-1],plot=False,disp=False)
    h=6.625e-34
    c=2.998e8
    OSS=3100
    row_min=10
    row_max=100
    col_min=1000
    
    """Get QE data from a file list.

    Parameters:
    -----------
    files : file list
    gain  : detector gain [ADU/e-]
    
    Returns
    -------
    data : summary data from images
    """
    abs_QE = np.zeros(shape=(hdus,len(proc_qe_files)))
    waves=list()
    power=list()
    
    for fz in raw_qe_files:
        hdr=fits.open(fz)[0].header
        waves.append(hdr['WAVE'])
        power.append(hdr['POWER'])
    
    power =np.array(power)*make_abs_calibration.get_calibration()
    
    for hdu in range(hdus):
        for i,f in enumerate(proc_qe_files):
            img=fits.open(f)
            os=sigma_clip(img[hdu].data[:,OSS:].flatten(),maxiters=None)
            act=sigma_clip(img[hdu].data[row_min:row_max,col_min:OSS].flatten(),maxiters=None)
            
            # Compute absolute QE
            
            mean_OS = np.mean(os)
            mean_ACT = np.mean(act)
            mean = mean_ACT-mean_OS
            
            # Convert from ADUs to elelctrons 
            mean_electron=mean/gain[hdu]
            abs_qe = mean_electron * ((h*c)/(power[i]*exp_time*(waves[i])/(1e9)))
            abs_QE[hdu][i]=abs_qe
    
    h,q,w=np.loadtxt('qe.txt',usecols=(0,1,2),unpack=True)
    flaugher = pd.read_csv('decam_qe_flaugher.csv',comment='#')
    font = FontProperties(size= 35)
    font.set_style('normal')

    Astro_5_qe = np.load('Astro_5_ABS_QE.npy')

    
    for hdu in range(hdus):
        fig, ax = plt.subplots(figsize=(13, 8))
        plt.plot(waves,abs_QE[hdu],'o',alpha=0.8,markersize=10,
                 color='green',label="AstroSkipper 8",
                 zorder=1,marker="X")
        plt.xlim(390,1200)
        plt.grid()
        ax.set_xlabel('Wavelength (nm)',fontproperties=font)
        ax.set_ylabel('Absolute QE',fontproperties=font)
        ax.tick_params(axis='both', which='major', labelsize=30)
        plt.legend(fontsize=20)
        plt.show()
calculate_abs_qe(qe_files,qe_files_fz)

# Diffusion

In [ ]:
def diffusion(files):
    '''
    Code from David Lawrence, Paul O’Connor, et al.,  
    “Model-independent Characterization of Charge Diffusion in Thick Fully Depleted CCDs”
    '''
    marker =["p","v","*",'X']
    psf_sigma=list()
    vsub=[30,40,50,60,70]
    hdus=4
    gain=get_gain(bckg[-1],plot=False,disp=False)
    for hdu in range(hdus):
        for f in files:
            img_xray= fits.open(f)
            img_xray=img_xray[hdu].data / gain[hdu]
            psf_sigma.append(dratio.analyze(img_xray[10:,1000:])[0])

    psf_sigma = np.array(psf_sigma).reshape(hdus,len(vsub),len(vsub))
    psf_sigma_mean = np.mean(psf_sigma,axis=2)
    
    fig, ax = plt.subplots(figsize=(8, 8)) 
    for hdu in range(hdus):
        plt.scatter(vsub,psf_sigma_mean[hdu],s=120,alpha=1 ,label = "HDU={}".format(hdu), marker=marker[hdu])
    
    ax.tick_params(axis='both', which='major', labelsize=25)
    plt.xlabel("VSUB",fontsize=40)
    plt.ylabel("PSF size (pixels)",fontsize=40)
    plt.legend(loc='best',fontsize=20)
    plt.grid()
    plt.show()
